# Convert the asymmetric adj to symmetric

- Time cost: several hours (?)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
from scipy.sparse import csr_matrix
import time
from sklearn.preprocessing import normalize


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
import sys
sys.path.append("../")
import utils

In [3]:
import importlib
importlib.reload(utils)

<module 'utils' from '../utils.py'>

In [4]:
with open("../../data/03_processed/A_home_activity_three_cities_unweighted_dic.pickle", 'rb') as f:
    A_home_activity_unweighted_dic = pickle.load(f)


In [5]:
def turn_asymmetric_directed_to_symmetric_undirected_adj(activity_dir_adj_df):
    ''' turn an asymmetric directed adj matrix to a symmetric directed one '''
#     activity_dir_adj_df = A_home_activity_unweighted_dic['boston'][1.0]['ATM']
    union_list = sorted(set(activity_dir_adj_df.columns).union(set(activity_dir_adj_df.index)))
    activity_zeros_df = pd.DataFrame(np.zeros((len(union_list), len(union_list))), 
                                     columns = union_list,
                                     index = union_list)
    activity_zeros_df = activity_zeros_df.add(activity_dir_adj_df, fill_value = 0.0)
#     print(activity_zeros_df)

    activity_undir_adj_df = pd.DataFrame(np.maximum(activity_zeros_df.values.T, activity_zeros_df.values),
                                   columns = union_list,
                                   index = union_list)
    return activity_undir_adj_df


# # test
# activity_dir_adj_df = A_home_activity_unweighted_dic['boston'][1.0]['ATM']
# activity_undir_adj_df = turn_asymmetric_directed_to_symmetric_undirected_adj(activity_dir_adj_df)
# print(np.sum(activity_undir_adj_df.values.T - activity_undir_adj_df.values)) # pass the test.
# # 0.0 is the right result.

0.0


In [ ]:
# initialize a new sym adj dic
# the file is HUGE
A_home_activity_unweighted_sym_adj_dic = {}

for city in ['boston','chicago','miami']:
    print(city)
    A_home_activity_unweighted_sym_adj_dic[city] = {}
    for threshold in [1.0, 10.0]:
        print(threshold)
        A_home_activity_unweighted_sym_adj_dic[city][threshold] = {}
        for activity_name in A_home_activity_unweighted_dic[city][threshold].keys():
            A_home_activity_unweighted_sym_adj_dic[city][threshold][activity_name]=turn_asymmetric_directed_to_symmetric_undirected_adj(A_home_activity_unweighted_dic[city][threshold][activity_name])


# Save

In [ ]:
# save - wow a huge file.
with open('../../data/03_processed/A_home_activity_three_cities_unweighted_sym_adj_dic.pickle', 'wb') as f:
    pickle.dump(A_home_activity_unweighted_sym_adj_dic, f)
